# Création de la base de données test

Relancer le kernel à chaque éxécution de ce fichier

In [1]:
import random
import numpy as np
import os
import shutil

In [2]:
shutil.rmtree("BD/apprentissage")
shutil.rmtree("BD/test")
shutil.rmtree("BD/modele")
shutil.rmtree("BD/prediction")

os.makedirs("BD/apprentissage")
os.makedirs("BD/test")
os.makedirs("BD/modele")
os.makedirs("BD/prediction")

Ce document va nous permettre de créer une base de test pour paramétrer les fonctions que nous allons utiliser

In [3]:
"""variables d'initialisation"""
nb_genre = 3
nb_ville = 5
nb_album = 700
tps_obs = 100
nb_block = 10 #nécessaire pour la cross validation
sig = 10 #valeur permettant de faire varier le bruit

#### Réalisation des matrices de transformations

Les matrices de transformations vont nous permettre de creer des vecteurs pour chaque temps de chaque albums en fonction du genre de l'album. On regroupe toutes les matrices de transformation sous un tableau nommé "mats" à 3 dimensions.

In [4]:
def matrice_transfo(k):
    """int -> [float*float]
       k est le nombre de villes
       retourne une matrice de transformation"""
    mat = np.zeros((k,k))
    for i in range(k):
        for j in range(k):
            mat[i][j] = random.uniform(-0.2, 0.2)
    return mat

In [5]:
def creation_matrice_transfo(ng, k):
    """ng est le nombre de genres
       k est le nombre de villes
       retourne l'ensemble des matrices de transformations"""
    mats = []
    for i in range(ng):
        mats.append(matrice_transfo(k))
    return np.array(mats)

In [6]:
mats = creation_matrice_transfo(nb_genre, nb_ville)

#### Réalisation de notre réprtoire d'albums

On commence par créer des vecteurs initiaux différents pour chaque genre d'album. Ce vecteur représente la répartition au temps 0 pour chaque ville.

In [7]:
def init_vecteur(k, tobs, genre, mats):
    """int*int -> [float*float]
       k est le nombre de villes
       tobs est le temps durant lequel on observe la diffusion
       genre est le type de musique de l'album
       mats est la matrice qui contient les matrices de transformation
       retourne une matrice contenant tous les vecteurs aux différents temps 'tobs' pour l'album 'a'
       (seul le vecteur au temps tobs=0 est remplit)"""
    v = np.zeros((tobs,k))
    mu = np.dot(np.ones(k), mats[genre-1])
    v[0] = np.random.normal(mu,sig,k)
    #on supprime les valeurs négatives
    v[0][v[0]<0] = 0
    return v

On créer un repertoire qui contient tous les albums à chaque temps et pour chaque ville.

In [8]:
def init_repertoire(a, k, tobs, mats, nb_genre):
    """int*int*int -> [float*float*float]*[int]
        a est le nombre d'albums
        k est le nombre de villes
        tobs est le temps durant lequel on observe la diffusion
        mats est la matrice qui contient les matrices de transformation
        nb_genre : nombre de label
        retourne un répertoire d'albums ainsi qu'un vecteur correspondant aux labels"""
    rep = []
    label = []
    for i in range(a):
        label.append(random.randrange(1,nb_genre+1))
        rep.append(init_vecteur(k, tobs, label[i], mats))
    return np.array(rep), np.array(label)

In [9]:
repertoire, labels = init_repertoire(nb_album, nb_ville, tps_obs, mats, nb_genre)

#### Remplissage du répertoire grâce aux matrices de transformations et à la loi normale

On applique la matrice de transformation associée au genre d'un album sur chaque temps pour obtenir le vecteur correspondant au temps t+1.

In [10]:
def transfo(v, mat):
    """[float]*[float*float] -> [float]
        v est un vecteur à un temps t
        mat est la matrice de transformation associé au label de ce vecteur
        retourne un nouveau vecteur au temps t+1"""
    mu = np.dot(v,mat)
    nv = np.random.normal(mu, sig)
    nv[nv<0] = 0
    return nv

In [11]:
def all_transfo(rep, labels, mats):
    """[float*float*flaot]*[float]*[float*float*float] -> [float*float*float]
       rep est un répertoire d'album
       labels est le tableau des labels associés aux albums
       mats est l'ensemble des matrices de transformations pour chaque genre de musique
       retourne un nouveau répertoire remplit pour chaque temps et chaque album"""
    #pour chaque album
    for ai in range(len(rep)):
        #pour chaque pas de temps
        for ti in range(1,len(rep[ai])):
            rep[ai][ti] = transfo(rep[ai][ti-1], mats[labels[ai]-1])
    return rep

In [12]:
repertoire = all_transfo(repertoire, labels, mats)

#### Creation des fichiers lisible par SVM light

##### * Ecriture

In [13]:
def ecriture_fichier(nom_fichier, repertoire, labels):
    """créé un fichier au format lisible par SVM light -> -1 1:0.43 3:0.12 9284:0.2 # abcdef
       dimension 1 : ville 1 au temps 1
       dimension 2 : ville 2 au temps 1
       ...
       dimension x : ville 1 au temps 2
       ...
       """
    fichier = open(nom_fichier, "w")
    tmp = ""
    for ai in range(len(repertoire)):
        dim = 1
        tmp = str(labels[ai])
        for ti in range(len(repertoire[ai])):
            for vi in range(len(repertoire[ai][ti])):
                tmp += " "+str(dim)+":"+str(repertoire[ai][ti][vi])
                dim += 1
        fichier.write(tmp+"\n")
        """if(ai!=len(repertoire)-1):
            fichier.write(tmp+"\n")
        else:
            fichier.write(tmp)"""

##### * Cross Validation

In [14]:
def separation(repertoire, labels, nb_block):
    """nb_block : nombre de bloks à créer à partir de notre repertoire
    fonction qui sépare le repertoire et les labels en 's' blocks"""
    liste_rep = []
    liste_labels = []
    s = len(repertoire)//nb_block
    
    for i in range(0,len(repertoire)-s+1, s):
        liste_rep.append(repertoire[i:i+s])
        liste_labels.append(labels[i:i+s])
    return liste_rep, liste_labels

In [15]:
def cross_validation(repertoire_block, labels_block):
    """repertoire_block : repertoire d'albums séparés en block
       labels_block : labels séparés en block associés à repertoire_block
       Fonction qui génére un ensemble de fichiers de tests et d'apprentissages sur le principe d'une cross validation"""
    for i in range(len(repertoire_block)):
        ecriture_fichier("BD/test/test"+str(i)+".txt", repertoire_block[i], labels_block[i])
        repertoire_appr = []
        labels_appr = []
        for j in range(len(repertoire_block)):
            if(i!=j):
                for k in range(len(repertoire_block[j])):
                    repertoire_appr.append(repertoire_block[j][k])
                    labels_appr.append(labels_block[j][k])
        ecriture_fichier("BD/apprentissage/apprentissage"+str(i)+".txt", repertoire_appr, labels_appr)

In [16]:
def creation_fichiers(repertoire, labels, s):
    """s : le nombre de block
       fonction créée l'ensemble des fichiers d'apprentissages et de tests"""
    rep, lab = separation(repertoire, labels, s)
    cross_validation(rep, lab)

In [17]:
creation_fichiers(repertoire, labels, nb_block)